In [ ]:
import os
import sys
import warnings
import itertools
import numpy as np
import pandas as pd
from collections import defaultdict

import sksurv
from sksurv.util import Surv
from sksurv.ensemble import RandomSurvivalForest
from sklearn.model_selection import train_test_split
from sksurv.metrics import integrated_brier_score
from sksurv.metrics import concordance_index_censored
from sklearn.model_selection import GridSearchCV


warnings.filterwarnings("ignore")

In [ ]:
sys.path[1:1] = ["/home/ec2-user/SageMaker/GitHub Repos/cipy/cipy/"]
sys.path[2:2] = ["/home/ec2-user/SageMaker/Users/SP056963/work_queue_prioritization_v2"]

sys.path

In [ ]:
from wqp_methods import *

In [ ]:
# Reading the Institutional claims dataset
ref_col= "claim_filing_ind_code3"

# csv_path for Institutional Claims Data Path
csv_path = "/home/ec2-user/SageMaker/Users/SP056963/work_queue_prioritization_v2/01_data_preprocessing/preprocessed_datasets/"

prof_claims= pd.read_csv(csv_path+"pc_preprocessed_dataset_2021-06-01.csv")

# Creating the event_flag - for survival analysis.
prof_claims["event_flag"] = True

In [ ]:
rfe_feat_path = "/home/ec2-user/SageMaker/Users/SP056963/work_queue_prioritization_v2/02_feat_sel_and_imp/professional_claims/03_step/pc_rfe_output.pickle"

with open(rfe_feat_path, "rb") as input_file:
    rfe_feat_op = pickle.load(input_file)

In [ ]:
# Claim filing indicator specific continuous and categorical features
commercial_num_feat = ["days_taken_for_claim_filing",
                       "total_claim_charge_amount",
                       "count_line_items"]

commercial_cat_feat = ["payer_state",
                       "payer_city",
                       "payer_name",
                       "claim_creation_month",
                       "claim_creation_quarter",
                       "facility_name",
                       "has_subgroup_or_policy_number",
                       "claim_creation_weekday"]

In [ ]:
# Performing outlier removal in response time values, encoding categorical data and then combining categorical and continuous features
payer_name = "Commercial Insurance Co."

df = prof_claims[prof_claims["claim_filing_ind_code3"]==payer_name].reset_index(drop=True)
df = clip_right_tail2(df, 0.99)
    
df["response_time"] = df["response_time"].astype("float")
folds = get_folds(df) 

In [ ]:
# Calling grid_search meth# Encoding the categorical data
from collections import defaultdict

encoder_dict = defaultdict(LabelEncoder)
cat_df = df[commercial_cat_feat]
cat_df = cat_df.astype("str")
cat_df = cat_df.apply(lambda x: encoder_dict[x.name].fit_transform(x))

num_df = df[commercial_num_feat]

rsf_df = pd.concat([num_df, cat_df], axis=1)

y = Surv.from_arrays(np.repeat(True, len(df)), df["response_time"].values)    od
c_index_values_df = grid_search(df, rsf_df, grid_search_params, folds)

In [ ]:
# Defining the grid search parameters
param_grid = {"n_estimators":[20, 50, 100, 200], 
              "max_features":["sqrt", "log2"], 
              "max_depth":[5, 10, 15, 20],
              "min_samples_split":[8, 10], 
              "min_samples_leaf":[4, 5]}

In [ ]:
# Create the GridSearch Object
grid_search_cv = GridSearchCV(estimator=RandomSurvivalForest(oob_score=True), 
                              param_grid=param_grid, 
                              cv=folds,
                              n_jobs=-1, 
                              verbose=10,
                              return_train_score=True, 
                              refit=True)

In [ ]:
# Fit the rsf algorithm on training data
grid_search_cv.fit(rsf_df, y)

In [ ]:
# Best Model parameters
pd.set_option("display.max_colwidth", -1)
results = pd.DataFrame(grid_search_cv.cv_results_)
results.head()

In [ ]:
# Best Test C-Index Parameters:
mask = results["rank_test_c_index"]=0
print("Best Parameters: ", results.loc[mask, "params"])

In [ ]:
import matplotlib.pyplot as plt

plt.plot([i for i in results.index], [i for i in results["mean_train_c_index"].values], label="Train Mean C-Index")
plt.plot([i for i in results.index], [i for i in results["mean_test_c_index"].values], label="Test Mean C-Index")

plt.legend()
plt.show()

In [ ]:
mask = results["rank_test_c_index"]=0
params =  results.loc[mask, "params"]
print(params)

In [ ]:
pickle.dump(pd.concat([cat_df, num_df], axis=1), open("TD_005010X222A1_"+payer_name+".pickle", "wb"))
pickle.dump(rsf_df, open("ED_005010X222A1_"+payer_name+".pickle", "wb"))
pickle.dump(encoder_dict, open("LE_005010X222A1_"+payer_name+".pickle", "wb"))
pickle.dump(rsf, open("005010X222A1_"+payer_name+".pickle", "wb"))
pickle.dump(results, open("RE_005010X222A1_"+payer_name+".pickle", "wb"))